<h1 style='font-size:40px'> Natural Language
Processing with RNNs and
Attention</h1>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O capítulo será composto por duas seções. Na primeira, iremos continuar desenvolvendo nossos estudos com RNN's, mas aplicadas no âmbito de NLP. Já na segunda, iremos dar enfoque aos mecanismos de atenção.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Generating Shakespearean Text Using a
Character RNN</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Aqui, vamos montar uma rede capaz de gerar poemas com o estilo de escrita de Shakespeare. Ela será treinada para prever o próximo caractere a ser digitado.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Creating the Training Dataset</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Antes de tudo, vamos baixar o corpus do projeto e associá-lo a uma variável.
        </li>
    </ul>
 </div>

In [6]:
# Baixando o arquivo com textos do Shakespeare. 
from tensorflow.keras.utils import get_file
file = "shakespeare.txt"
url = "https://homl.info/shakespeare"
filepath = get_file(file, url)

In [7]:
with open(filepath, 'r') as f:
    texts = f.read()

In [8]:
# Para numeralizarmos os textos, vamos recorrer à classe `Tokenizer`. Ao setarmos `char_level=True`, vamos associar cada caractere do corpus
# a um número (começando por 1). 
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([texts])

In [9]:
# Tokenização de caracteres.
tokenizer.texts_to_sequences(['Hi, there!'])

[[7, 6, 18, 1, 3, 7, 2, 9, 2, 31]]

In [10]:
# Observe que o objeto não considera caracteres acentuados, por padrão.
len(tokenizer.texts_to_sequences(['Olá, como vai?'])[0]), len('Ola, como vai?')

(13, 14)

In [11]:
# Convertendo índices em uma string.
tokenizer.sequences_to_texts([[1,2,3,4,5]])

['  e t o a']

In [12]:
# Atribuindo os índices do texto à variável encoded. Vamos fazer uma subtração para que o primeiro índice seja atribuído a 0.
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([texts])) - 1
encoded

array([19,  5,  8, ..., 20, 26, 10])

<h3 style='font-size:30px;font-style:italic'> How to Split a Sequential Dataset</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Vamos dividir aqui o nosso corpus no set de treino, validação e teste. O autor aproveita a situação, e revela os prós e contras das diferentes metodologias de separação do dataset. 
        </li>
        <li>
            No nosso caso, vamos optar por manter os primeiros fragmentos do corpus para treinamento, e o restante para validação e teste. Isso tem a assunção de que os mesmos padrões presentes em momentos anteriores do arquivo estarão contidos em momentos futuros (série estacionária).
        </li>
    </ul>
 </div>

In [13]:
from tensorflow.data import Dataset
train_size = encoded.shape[0] * 90 // 100
dataset = Dataset.from_tensor_slices(encoded[:train_size])

<h3 style='font-size:30px;font-style:italic'>Chopping the Sequential Dataset into Multiple Windows</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Até agora, convertemos todo o texto em uma única instância. Como treinar um modelo com isso é inviável, temos que quebrar esses dados em janelas, tratando cada uma delas como uma instância.
        </li>
        <li>
            A função `Dataset.window` cria datasets dentro de nosso dataset principal, cada um contendo uma porção específica de elementos. 
        </li>
    </ul>
 </div>

In [14]:
# Criando sub-datasets de 101 caracteres. Quando definimos `shift=1`, escolhemos que a janela se desloque 1 caractere por vez para montar 
# cada sub-dataset [0-101, 1-102...].

n_steps = 100
window_length = n_steps+1
# `drop_remainder` exclui as últimas janelas cujo tamanho acabaria menor do que `size`.
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

<div> 
    <ul style='font-size:20px'> 
        <li> 
            No entanto, devemos observar que as classes de modelo admitem apenas `tf.Tensor` como input. Caso quisermos converter cada Dataset aninhado em um tensor, podemos fazer um truque com a função `flat_map`, passando a ela uma lambda que cria batches de mesmo comprimento que `window_length`.
        </li>
        <li>
            Em tese, `flat_map` torna Datasets de aninhamento num único Dataset. Mas, como ele admite uma função que aplique alguma transformação nos Datasets aninhados antes da planificação, podemos aplicar `batch`. No final das contas, ficamos com tensores de mesmo conteúdo que os Datasets.
        </li>
    </ul>
 </div>

In [15]:
dataset = dataset.flat_map(lambda w: w.batch(window_length))

In [16]:
# Vamos aproveitar a situação, e já criar batches com vários chunks aleatórios.
dataset = dataset.shuffle(1000).batch(32)

In [17]:
# Criando o X e y da IA. Vamos programá-la para que ela receba a sentença recortada do primeiro ao penúltimo caractere,
# e preveja a sequência do segundo ao último caractere.
dataset = dataset.map(lambda w: (w[:, :-1], w[:, 1:]))

In [18]:
# Vamos tornar o X num One-Hot Encoding.
from tensorflow import one_hot

max_id = len(tokenizer.word_index) # Quantidade de caracteres distintos.
n = dataset.map(lambda X,y: (one_hot(X, depth=max_id), y)).as_numpy_iterator()

In [19]:
# Garantindo que o próximo batch a ser processado no treinamento já seja alocado à memória, antecipadamente.
dataset.prefetch(1)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

<h3 style='font-size:30px;font-style:italic'>Building and Training the Char-RNN Model</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Vamos montar agora uma rede recorrente que preverá o próximo caractere de uma sequência.
        </li>
        <li>
            Não montei a rede, porque demora muito para treinar.
        </li>
    </ul>
 </div>

In [20]:
import numpy as np
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

<h3 style='font-size:30px;font-style:italic'>Generating Fake Shakespearean Text</h3>
<div> 
    <ul style='font-size:20px'> 
        <li>
            A geração de textos completos poderia acontecer em ciclos, em que dada uma frase, o modelo prevê o próximo caractere e o vincula ao final da frase para uma nova previsão. O problema dessa abordagem é a alta probabilidade de sempre o mesmo caractere ser previsto...
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Temperature</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Uma maneira de garantirmos outputs mais diversos é acrescentar algumas etapas pós-predict do modelo. Tendo as probabilidades, extraímos o seu log e as dividimos por uma `temperature`. O valor desse argumento é de escolha do desenvolvedor.
        </li>
        <li>
            Nós usaremos a função `tf.random.categorical`, que escolherá o próximo token da frase, dado os logits computados.
        </li>
        <li>
            Menores temperatures ($<1$) acentuam as diferenças entre os logits, enquanto valores acima de 1 tendem a planificar esse conjunto de números. 
        </li>
    </ul>
 </div>

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [22]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

<h3 style='font-size:30px;font-style:italic'> Statefull RNN</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Uma RNN Stateful é aquela que preserva o seu state anterior para a próxima iteração de treinamento. No caso, isso é apenas recomendado caso a primeira instância do próximo batch seja uma continuação da última do batch anterior.
        </li>
        <li>
            No entanto, quando fazemos batch, as instâncias não são consecutivas por posição. No caso de $\text{batch}=32$, as primeiras instâncias dos dois primeiros batches (1 e 33) não são consecutivas.
        </li>
        <li>
            A solução para isso seria criar batches unitários!
        </li>
    </ul>
 </div>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, TimeDistributed

# Deixe `stateful=True` em cada uma de suas camadas recorrentes.
# Informe `batch_input_shape` na primeira camada da rede.
stateful_model = Sequential([
    GRU(128, return_sequences=True, stateful=True, dropout=.2, recurrent_dropout=.2, batch_input_shape=[batch_size, None, max_id]),
    GRU(128, return_sequences=True, stateful=True, dropout=.2, recurrent_dropout=.2),
    TimeDistributed(Dense(max_id, activation='softmax'))
])

<p style='color:red'> Iniciei Stateful RNN's; Montar a classe ResetStatesCallback; Pesqui </p>